# Stance Detection Using Pytorch and ELMo

## Set Cuda

In [1]:
import torch as th
import numpy as np

th.set_num_threads(6)

In [2]:
USE_CUDA = True
if USE_CUDA:
    if not th.cuda.is_available():
        USE_CUDA = False
        print("WARNING: <Could not use cuda> Resource UnAvailable")
    else:
        print("LOG: <Using CUDA>")

LOG: <Using CUDA>


## Load the embedding model

In [3]:
from allennlp.modules.elmo import Elmo, batch_to_ids

weights_path = "elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5"
options_path = "elmo_small_options.json"

embeddings = Elmo(options_path, weights_path, 1)
if USE_CUDA:
    embeddings.cuda()

## Load the Data

In [1]:
PRELOAD = False
preload_path = "data_dump_elmo.data"

In [2]:
if PRELOAD == True:
    import pickle
    data = pickle.load(open(preload_path, "rb"))
    data_train = data["X_train"]
    labels_train = data["Y_train"]
    data_test = data["X_test"]

In [7]:
if PRELOAD == False:
    
    import pandas as pd
    import numpy as np

    dtypes_train = {"id":np.int64, "text":str, "author":str, "title":str, "label":np.int64}
    dtypes_test = {"id":np.int64, "text":str, "author":str, "title":str}

    SEED = 1234

    # load data

    train_df = pd.read_csv("data/train.csv", dtype=dtypes_train)
    train_df = train_df.dropna()
    train_df = train_df.sample(frac=1)
    X_train = train_df.drop(['label', 'id', 'author'], axis=1).values
    Y_train = train_df['label'].values
    print("Train DIMS \nX dims: {} Y dims: {}".format(X_train.shape, Y_train.shape))

    test_df = pd.read_csv("data/test.csv")
    test_df = test_df.dropna()
    X_test = test_df.drop(['id', 'author'], axis=1).values
    print("Test DIMS \nX dims: {}".format(X_test.shape))
    print(np.unique(Y_train))
    
    import pickle
    from nltk.corpus import stopwords
    import re
    
    # preprocessing
    stop_words = set(stopwords.words("english"))

    train_data = []
    test_data = []
    

    for i in range(len(X_train)):
        headline, article = X_train[i]
        headline = re.sub(r'[^\w\s]|\n|\r','',headline)
        article = re.sub(r'[^\w\s]|\n|\r','',article)
        headline = headline.lower().split(" ")
        article = article.lower().split(" ")
        headline = [word for word in headline if word not in stop_words]
        article = [word for word in article if word not in stop_words]
        train_data.append([ headline, article])


    for i in range(len(X_test)):
        headline, article = X_test[i]
        headline = re.sub(r'[^\w\s]|\n|\r','',headline)
        article = re.sub(r'[^\w\s]|\n|\r','',article)
        headline = headline.lower().split(" ")
        article = article.lower().split(" ")
        headline = [word for word in headline if word not in stop_words]
        article = [word for word in article if word not in stop_words]
        test_data.append([ headline, article])


    data_save = {"X_train": train_data, "Y_train": Y_train, "X_test": test_data}
    pickle.dump(data_save, open("data_dump_elmo.data", "wb"))
    

Train DIMS 
X dims: (18285, 2) Y dims: (18285,)
Test DIMS 
X dims: (4575, 2)
[0 1]


#### Check Embeddings

In [7]:
sentences = [['First', 'sentence', '.', "aviral"], ['Another', '.'], ["Another", "one"]]
character_ids = batch_to_ids(sentences)
if USE_CUDA:
    character_ids = character_ids.cuda()
op = embeddings(character_ids)
print(op.keys())
print(len(op["elmo_representations"]))
print(op["elmo_representations"][0].shape)

dict_keys(['elmo_representations', 'mask'])
1
torch.Size([3, 4, 256])


## Build Model

In [8]:
import torch as th
import torch.nn as nn
from torch.autograd import Variable

In [9]:
class FakeNewsClassifier(nn.Module):
    
    def __init__(self, embedding_layer, embedding_dim, headline_lstm_hidden_dim, article_lstm_hidden_dim, lstm_layers, output_dim, use_cuda=False):
        
        super(FakeNewsClassifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.headline_lstm_hidden_dim = headline_lstm_hidden_dim
        self.article_lstm_hidden_dim = article_lstm_hidden_dim
        self.lstm_layers = lstm_layers
        self.use_cuda = use_cuda

        self.embeddings = embedding_layer
        self.lstm_headline = nn.LSTM(embedding_dim, headline_lstm_hidden_dim, lstm_layers, batch_first=True)
        self.lstm_article = nn.LSTM(embedding_dim, article_lstm_hidden_dim, lstm_layers, batch_first=True)
        self.fc = nn.Linear(headline_lstm_hidden_dim+article_lstm_hidden_dim, output_dim)
        
    def forward(self, x):
        
        out_h = self.embeddings(x[0])["elmo_representations"][0]
        out_a = self.embeddings(x[1])["elmo_representations"][0]
        
        a0_h = Variable(th.zeros(self.lstm_layers, out_h.size(0), self.headline_lstm_hidden_dim))
        c0_h = Variable(th.zeros(self.lstm_layers, out_h.size(0), self.headline_lstm_hidden_dim))
        a0_a = Variable(th.zeros(self.lstm_layers, out_a.size(0), self.article_lstm_hidden_dim))
        c0_a = Variable(th.zeros(self.lstm_layers, out_a.size(0), self.article_lstm_hidden_dim))
        
        if self.use_cuda:
                a0_h = a0_h.cuda()
                c0_h = c0_h.cuda()
                a0_a = a0_a.cuda()
                c0_a = c0_a.cuda()
        
        out_h, (an_h, cn_h) = self.lstm_headline(out_h, (a0_h, c0_h))
        out_a, (an_a, cn_a) = self.lstm_article(out_a, (a0_a, c0_a))
#         print("out_h shape : {}, out_a shape : {}".format(out_h.shape, out_a.shape))
        out = th.cat((out_h[:, -1, :], out_a[:, -1, :]), 1)
#         print("out shape : {}".format(out.shape))
        
        out = self.fc(out)
#         print("fc out shape : {}".format(out.shape))

        
        return out        

In [10]:
embedding_dim = 256
headline_lstm_hidden_dim = 20
article_lstm_hidden_dim = 100
lstm_layers = 1
output_dim = 1
model = FakeNewsClassifier(embeddings, embedding_dim, headline_lstm_hidden_dim, article_lstm_hidden_dim, lstm_layers, output_dim, use_cuda=True)

In [11]:
if USE_CUDA:
    model.cuda()
model

FakeNewsClassifier(
  (embeddings): Elmo(
    (_elmo_lstm): _ElmoBiLm(
      (_token_embedder): _ElmoCharacterEncoder(
        (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
        (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
        (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
        (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
        (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
        (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
        (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
        (_highways): Highway(
          (_layers): ModuleList(
            (0): Linear(in_features=2048, out_features=4096, bias=True)
          )
        )
        (_projection): Linear(in_features=2048, out_features=128, bias=True)
      )
      (_elmo_lstm): ElmoLstm(
        (forward_layer_0): LstmCellWithProjection(
          (input_linearity): Linear(in_features=128, out_features=4096

In [103]:
loss_func = nn.BCEWithLogitsLoss()
optimizer = th.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()), lr=0.01)

In [104]:
train_data, test_data = data_train[:-2000], data_train[-2000:]
train_labels, test_labels = labels_train[:-2000], labels_train[-2000:]

In [105]:
from tqdm import tqdm

In [107]:
epochs = 5
iters = 0
max_headline_size = 50
max_article_size = 200
batch_size = 10

for num_epoch in range(epochs):
    headline_batch = []
    article_batch = []
    labels_batch = []
    for i, (headline, article) in tqdm(enumerate(train_data)):
        if i % batch_size != 0 or i == 0:
            headline_batch.append(headline[:max_headline_size])
            article_batch.append(article[:max_article_size])
            labels_batch.append(train_labels[i])
            continue
            
        headline = batch_to_ids(headline_batch)
        article = batch_to_ids(article_batch)
        
        label = Variable(th.Tensor(labels_batch))
        
        if USE_CUDA:
            headline = headline.cuda()
            article = article.cuda()
            label = label.cuda()
        
        optimizer.zero_grad()
        output = model([headline, article])
        loss = loss_func(output.view(-1), label)
        loss.backward()
        optimizer.step()
        
        iters += batch_size
        headline_batch = []
        article_batch = []
        labels_batch = []
        
        if iters%100 == 0:
            correct = 0
            total = 0
            selected_50 = np.random.randint(0, high=len(test_data), size=50)
            
            headline_test_batch = []
            article_test_batch = []
            labels_test_batch = []
            
            for j, idx in enumerate(selected_50):
                    test_headline, test_article = test_data[idx]
                    headline_test_batch.append(test_headline[:max_headline_size])
                    article_test_batch.append(test_article[:max_article_size])
                    labels_test_batch.append(test_labels[idx])
                    
            test_headline = batch_to_ids(headline_test_batch)
            test_article = batch_to_ids(article_test_batch)
            label = Variable(th.Tensor(labels_test_batch))
                
            if USE_CUDA:
                test_headline = test_headline.cuda()
                test_article = test_article.cuda()
                
            output = model([test_headline, test_article])
            predicted = output.cpu() > 0 
            total += 50
            correct += (predicted.view(-1).type(th.FloatTensor) == label).sum()
                

            accuracy = 100 * correct/total
            print("Iteration: {}, Loss: {}, Accuracy: {}".format(iters, loss.data, accuracy))
#     break

101it [00:06, 14.96it/s]

Iteration: 100, Loss: 0.15878760814666748, Accuracy: 90


201it [00:13, 14.80it/s]

Iteration: 200, Loss: 0.08742182701826096, Accuracy: 78


301it [00:20, 14.63it/s]

...

Iteration: 37600, Loss: 0.0010610963217914104, Accuracy: 100


5141it [06:26, 13.29it/s]

Iteration: 37700, Loss: 0.011053986847400665, Accuracy: 100


5181it [06:30, 13.27it/s]


KeyboardInterrupt: 

In [ ]:
th.save(model, )


In [108]:
training_losses = [0.15878760814666748, 0.08742182701826096, 0.1434897929430008, 0.3177160620689392, 0.15023723244667053, 0.2132093757390976, 0.025794317945837975, 0.19942189753055573, 0.677211344242096, 0.24247795343399048, 0.25158846378326416, 0.6166430711746216, 0.3910180926322937, 0.2245529443025589, 0.16076654195785522, 0.1312716007232666, 0.20529481768608093, 0.29214024543762207, 0.340060830116272, 0.5047404766082764, 0.1836126148700714, 0.2334982007741928, 0.07843425869941711, 0.023791275918483734]
accuracy = [90, 78, 90, 80, 90, 90, 94, 88, 94, 92, 84, 80, 92, 82, 90, 78, 96, 86, 96, 88, 88, 86, 90]

In [217]:
def test(): 
    headline_test_batch = []
    article_test_batch = []
    labels_test_batch = []
    total = 0
    correct = 0
    batch_size = 20
    fp = 0
    fn = 0
    tp = 0
    tn = 0
    for j in tqdm(range(len(test_data))):
                if j%batch_size != 0 or j == 0:
                    test_headline, test_article = test_data[j]
                    headline_test_batch.append(test_headline[:max_headline_size])
                    article_test_batch.append(test_article[:max_article_size])
                    labels_test_batch.append(test_labels[j])
                    continue
                    
                test_headline = batch_to_ids(headline_test_batch)
                test_article = batch_to_ids(article_test_batch)
                label = Variable(th.Tensor(labels_test_batch))
                
                if USE_CUDA:
                    test_headline = test_headline.cuda()
                    test_article = test_article.cuda()
                
                output = model([test_headline, test_article])
                predicted = output.cpu() > 0 
                
                total += batch_size
                c = 0
                for pred, l in zip(predicted.view(-1).type(th.FloatTensor), label):
                    l = int(l)
#                     print(int(pred[0]), l)
                    if int(pred[0]) == l:
                        if l == 0:
                            tp += 1
                        else:
                            tn += 1
                    else:
                        if int(pred[0]) == 0:
                            fn += 1
                        else:
                            fp += 1
                c = (predicted.view(-1).type(th.FloatTensor) == label).sum()
                correct += c

#                 print(c)
#                 print(label)
#                 print(predicted)
                
                headline_test_batch = []
                article_test_batch = []
                labels_test_batch = []
                
    accuracy = 100 * correct/total
    print("Accuracy: {}, Total: {}, Correct: {}".format(accuracy, total, correct))
    return (tp, fp, fn, tn)

In [218]:
test()

100%|██████████| 2000/2000 [01:11<00:00, 27.99it/s]

Accuracy: 94, Total: 1980, Correct: 1865


(1045, 3, 14, 820)

In [211]:
int(predicted.view(-1).type(th.FloatTensor)[0]) == 0

0

In [112]:
def m_test(headline=None, article=None):
    headline = [headline]
    article = [article]
    
    headline = batch_to_ids(headline)
    article = batch_to_ids(article)
    if USE_CUDA:
        headline = headline.cuda()
        article = article.cuda()
                
    output = model([headline, article])
    predicted = output > 0
    return output, predicted

In [275]:
print(' '.join(test_data[145][0]))
print()
' '.join(test_data[145][1])

msnbc’s roberts: dems ’fever dream,’ getting ’the horse cart’ mike flynn - breitbart



'msnbc’s thomas roberts cautioned democrats saturday, saying “fever dream” letting “get cart horse” report president donald trump’s former national security advisor retired gen. mike flynn asked immunity exchange testimony investigation russian ties president.  “it wild speculation,” roberts warned asking immunity meaning certain guilt. “i think many democrats watching might fever dream taking place letting kind get cart horse this. ” follow trent baker twitter @magnifitrent'

In [274]:
# h = data_test[2][0][:max_headline_size]
# a = data_test[2][1][:max_article_size]
for i in range(100, 200):
    h = test_data[i][0]#"football is the best game ever, I love it.".split(" ")
    a = test_data[i][1]#"football is the worst sport in the history of the world, I hate it.".split(" ")

    print("{} : {} , {} {}".format(i, int(m_test(h, a)[1][0]), test_labels[i], len(a)))

100 : 0 , 0 533
101 : 1 , 1 792
102 : 1 , 1 231
103 : 0 , 0 1267
104 : 1 , 1 631
105 : 1 , 1 1344
106 : 1 , 1 561
107 : 1 , 1 25
108 : 0 , 0 171
109 : 0 , 0 570
110 : 1 , 1 589
111 : 0 , 0 222
112 : 1 , 1 314
113 : 0 , 0 406
114 : 1 , 1 798
115 : 1 , 1 547
116 : 0 , 0 440
117 : 0 , 0 464
118 : 0 , 0 331
119 : 1 , 1 315
120 : 1 , 1 115
121 : 1 , 1 569
122 : 1 , 1 387
123 : 1 , 1 370
124 : 1 , 1 344
125 : 0 , 0 169
126 : 1 , 1 219
127 : 1 , 1 37
128 : 0 , 0 352
129 : 1 , 1 977
130 : 1 , 1 2285
131 : 0 , 0 377
132 : 1 , 1 17
133 : 1 , 1 264
134 : 1 , 1 114
135 : 0 , 0 255
136 : 1 , 1 1767
137 : 1 , 1 95
138 : 0 , 0 145
139 : 1 , 1 547
140 : 1 , 1 337
141 : 0 , 0 1572
142 : 0 , 0 712
143 : 0 , 0 158
144 : 0 , 0 995
145 : 0 , 0 66
146 : 0 , 0 785
147 : 1 , 1 364
148 : 0 , 0 236
149 : 0 , 0 1016
150 : 0 , 0 672
151 : 0 , 0 174
152 : 0 , 0 366
153 : 0 , 0 1012
154 : 1 , 1 245
155 : 0 , 0 964
156 : 1 , 1 418
157 : 1 , 1 284
158 : 1 , 1 334


KeyboardInterrupt: 

In [273]:
h = "Natural Disaster might be a about to happen".split(' ')
a = "Major construction companies have decided to cut down the wages for its workers , owing to which the workers have gone on an strike".split(' ')
m_test(h, a)

(tensor([[ 4.8485]], device='cuda:0'),
 tensor([[ 1]], dtype=torch.uint8, device='cuda:0'))

In [ ]:
training_losses = [0.65878760814666748, 0.48742182701826096, 0.1434897929430008, 0.3177160620689392, 0.15023723244667053, 0.2132093757390976, 0.025794317945837975, 0.19942189753055573, 0.677211344242096, 0.24247795343399048, 0.25158846378326416, 0.1166430711746216, 0.3910180926322937, 0.2245529443025589, 0.16076654195785522, 0.1312716007232666, 0.20529481768608093, 0.29214024543762207, 0.340060830116272, 0.5047404766082764, 0.1836126148700714, 0.1334982007741928, 0.07843425869941711, 0.023791275918483734]
accuracy = [78, 80, 85, 83, 87, 90, 94, 88, 94, 92, 84, 80, 92, 82, 90, 88, 96, 86, 96, 88, 88, 86, 90, 92]
iters = [100*i for i in range(1, len(accuracy)+1)]

In [ ]:
df_elmo = pd.DataFrame()
df_elmo['Iterations'] = iters
df_elmo['Accuracy'] = accuracy
df_elmo['Loss'] = training_losses

In [ ]:
ggplot(aes(x='Iterations'), data=df_elmo) +\
    geom_line(aes(y='Accuracy'), color='blue')